In [1]:
#Лабораторна робота № 7 Масалітін Ілля Вадимович ФІТ 4-8

In [2]:
from surprise import Dataset

data = Dataset.load_builtin('ml-100k')

trainset = data.build_full_trainset()
for i, (uid, iid, rating) in enumerate(trainset.all_ratings()):
    if i >= 5:
        break
    print(f"User: {uid}, Item: {iid}, Rating: {rating}")


User: 0, Item: 0, Rating: 3.0
User: 0, Item: 528, Rating: 4.0
User: 0, Item: 377, Rating: 4.0
User: 0, Item: 522, Rating: 3.0
User: 0, Item: 431, Rating: 5.0


In [3]:
from surprise import KNNBasic, SVD
from surprise.model_selection import cross_validate

knn = KNNBasic()
knn_results = cross_validate(knn, data, measures=['MAE'], cv=5, verbose=True)

svd = SVD()
svd_results = cross_validate(svd, data, measures=['MAE'], cv=5, verbose=True)


Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
MAE (testset)     0.7689  0.7753  0.7759  0.7741  0.7706  0.7730  0.0027  
Fit time          0.35    0.41    0.37    0.37    0.37    0.37    0.02    
Test time         1.81    1.81    1.80    1.82    1.81    1.81    0.01    
Evaluating MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
MAE (testset)     0.7354  0.7425  0.7390  0.7349  0.7418  0.7387  0.0031  
Fit time          0.75    0.76    0.78    0.77    0.76    0.76    0.01    
Test time

In [4]:
from surprise.model_selection import GridSearchCV

param_grid_knn = {
    'k': [20, 30],  
    'min_k': [1],  
    'sim_options': {
        'name': ['pearson'],  
        'user_based': [True]  
    }
}
gs_knn = GridSearchCV(KNNBasic, param_grid_knn, measures=['mae'], cv=5)
gs_knn.fit(data)
print("Найкращі параметри для KNN:", gs_knn.best_params['mae'])

param_grid_svd = {'n_factors': [50, 100], 'reg_all': [0.02, 0.1], 'lr_all': [0.005, 0.01]}
gs_svd = GridSearchCV(SVD, param_grid_svd, measures=['mae'], cv=5)
gs_svd.fit(data)
print("Найкращі параметри для SVD:", gs_svd.best_params['mae'])


Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Найкращі параметри для KNN: {'k': 30, 'min_k': 1, 'sim_options': {'name': 'pearson', 'user_based': True}}
Найкращі параметри для SVD: {'n_factors': 100, 'reg_all': 0.1, 'lr_all': 0.01}


In [5]:
if gs_knn.best_score['mae'] < gs_svd.best_score['mae']:
    best_algorithm = 'KNN'
    best_model = gs_knn.best_estimator['mae']
else:
    best_algorithm = 'SVD'
    best_model = gs_svd.best_estimator['mae']

print(f"Найкращий алгоритм: {best_algorithm}")


Найкращий алгоритм: SVD


In [7]:
trainset = data.build_full_trainset()
best_model.fit(trainset)

user_id = str(196)
items = trainset.all_items()
items_rated_by_user = [item for (item, rating) in trainset.ur[trainset.to_inner_uid(user_id)]]

recommendations = []
for item in items:
    if item not in items_rated_by_user:
        recommendations.append((item, best_model.predict(user_id, trainset.to_raw_iid(item)).est))
        
recommendations.sort(key=lambda x: x[1], reverse=True)
top_10_recommendations = recommendations[:10]

print("Топ 10 рекомендацій:")
for item, rating in top_10_recommendations:
    print(f"Item: {trainset.to_raw_iid(item)}, Estimated Rating: {rating}")


Топ 10 рекомендацій:
Item: 1449, Estimated Rating: 4.469325795800039
Item: 64, Estimated Rating: 4.4393062006881845
Item: 318, Estimated Rating: 4.422667198082842
Item: 408, Estimated Rating: 4.377413362388025
Item: 169, Estimated Rating: 4.373853944999849
Item: 603, Estimated Rating: 4.325909394052005
Item: 12, Estimated Rating: 4.311569942178592
Item: 114, Estimated Rating: 4.28155419657698
Item: 483, Estimated Rating: 4.2729235974288065
Item: 178, Estimated Rating: 4.251824674720407


In [ ]:
#2

In [11]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

ratings = pd.read_csv('D:\\recomendations.csv')

user_item_matrix = ratings.pivot(index='userId', columns='movieId', values='rating')

user_similarity = cosine_similarity(user_item_matrix.fillna(0))

user_id = 1
user_ratings = user_item_matrix.loc[user_id].dropna()
predicted_ratings = (user_similarity[user_id-1] @ user_item_matrix.fillna(0)) / user_similarity[user_id-1].sum()

recommended_items = predicted_ratings[user_item_matrix.loc[user_id].isna()].nlargest(10)
print(recommended_items)


movieId
10    0.0
32    0.0
dtype: float64


In [ ]:
#Висновки: На основі датасету ml-100k ми реалізували алгоритми KNNBasic та SVD, також ми виявили що алгоритм SVD являється найбіль еффективним у роботі
#з великими матрицями.Рекомендацыйна система була створена на основы косынусної схожості.